In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import normalize

In [4]:
### Extraccion archivo desde google drive, configuracion general (directorios y parámetros)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DATA_DIR = "/content/drive/MyDrive/data"
TRAIN_DIR = f"{DATA_DIR}/train"
VAL_DIR = f"{DATA_DIR}/test"
IMG_PATH = f"{DATA_DIR}/soir_bleu.jpg"
EMBED_PATH = f"{DATA_DIR}/soir_bleu_embedding.npy"

In [7]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

### Estrategia TPU

In [8]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("✅ TPU activado")
except ValueError:
    strategy = tf.distribute.get_strategy()
    print("⚠️ TPU no encontrado, usando CPU/GPU")

⚠️ TPU no encontrado, usando CPU/GPU


### Dataset mediante carga de datos

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Clases: {class_names}")

In [ ]:
# Preprocesar
AUTOTUNE = tf.data.AUTOTUNE
def preprocess(image, label):
    return preprocess_input(image), label

train_ds = train_ds.map(preprocess).prefetch(AUTOTUNE)
val_ds = val_ds.map(preprocess).prefetch(AUTOTUNE)

### Callbacks

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
checkpoint = ModelCheckpoint("efficientnet_tuneaito.h5", monitor='val_loss', save_best_only=True)

### Declaramos la inicializacion del modelo

In [ ]:
with strategy.scope():
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False, weights='imagenet', input_shape=(224, 224, 3)
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

### Primera parte del entrenamiento (base)

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=[early_stop, reduce_lr, checkpoint])

### Aplicamos fine-tuning

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
  layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch=5,
    epochs=15,
    callbacks=[early_stop, reduce_lr, checkpoint]
)

In [ ]:
model = tf.keras.models.load_model("efficientnet_tuneaito.h5")
embedding_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-3].output)

In [ ]:
embedding_model.save(f"{DATA_DIR}/efficientnet_tuneaito.h5")

### PUESTA A PRUEBA DEL MODELO
 Extraccion de características desde el modelo mejorado con el nuevo dataset

In [ ]:
img = load_img(IMG_PATH, target_size=IMG_SIZE)
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

embedding = embedding_model.predict(img_array)
embedding = normalize(embedding)

np.save(EMBED_PATH, embedding)
print(f"✅ Embedding guardado en: {EMBED_PATH}")